# Script for master's thesis

This script contains the programming of all the models included in the thesis "...."

The script consist of the following: 
- Loading all the packages
- Importing data
- Defining ROC plot
- Logistsic regression 
    - Standard logistic regression
    - Ridge regression
    - Lasso regression
- Random forest
    - Variable importance
- Neural network
    - Neural network model 1
    - Neural network model 2


## Loading all the packages 

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics
from sklearn.datasets import make_classification
seed_number = 1000
print_flag = 1
plot_flag = 1


import plotly.express as px
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

## Importing data

In [ ]:
# Importing the dataset
df=pd.read_excel(r'C:\Users\cullu\OneDrive - CBS - Copenhagen Business School\Speciale\Output_final.xlsx')
print(df)

# Removing the columns not needed from the dataset
useless = ["recordID","start_date", "end_date", "postal_code","premium","value","brand", "age_group", "seniority_group", "hp_group", "fuel_group"]
DF_without_cats = df.drop(useless, axis=1)

# Adding dummies for 'age', 'seniority', 'hp', and 'fuel'
dummy_df=pd.get_dummies(DF_without_cats,columns=["age","seniority","hp", "fuel"])
#print(dummy_df)


# Splitting dataset into two sets, X for the predictors and y for the response variable                                
X = dummy_df.drop(["status"], axis = 1)
y = dummy_df["status"]

# Splitting X and y into training dataset and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed_number)


## Defining ROC plot

In [ ]:
def roc_plot(values, colors, h=10, w=7, lab=None, EKSTRA_TEXT = ""):
    sns.set()
    fig, ax = plt.subplots(figsize=(h, w))
    
    colors = colors
    i=0
    for val in values:
        ax.plot(val[1], val[0], label = '{} - AUC = {}'.format(lab[i], val[2]), color=colors[i])
        i += 1
        
    ax.plot(np.linspace(0, 1, 30), np.linspace(0, 1, 30), label='baseline', linestyle='--', color='orange')
    plt.title('Receiver Operating Characteristic' + EKSTRA_TEXT, fontsize=18)
    plt.ylabel('True Positive Rate', fontsize=16)
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylim(0, 1.05)
    plt.legend(loc = 'lower right', fontsize=12)
    
    return values, colors, lab

## Logistic regression

For logistsic regression we have computed three models: 
1. Standard logistic regression
2. Ridge regression
3. Lasso regression

### Standard logistic regression

In [ ]:
##### PLAIN #####

# Use the best regularization strength parameter
# optima parameter indsat
PLR_Model_1 = LogisticRegression()
####################################################################################################
                                        # grid search
# ####################################################################################################

parameters = {
   'solver': ['newton-cg', 'lbfgs', 'sag' and 'saga'],
     'penalty' : ['none']
            }
PLR_model_Grid_1 = GridSearchCV(    PLR_Model_1, 
                                         parameters, 
                                         cv=5, 
                                         scoring = "roc_auc", 
                                         # verbose=10, 
                                         n_jobs = -1
                                         )
PLR_model_Grid_1.fit(X_train, y_train)
print(sorted(PLR_model_Grid_1.cv_results_.keys()))

PLR_Model_1 = PLR_model_Grid_1
# ####################################################################################################
# ####################################################################################################

# Training the Random Forest model
PLR_Model_1.fit(X_train, y_train)
# print(datetime.now() - t12)

print(PLR_Model_1.best_estimator_)
print("Tuned Hyperparameters :", PLR_Model_1.best_params_)

# Model accuracy
if print_flag == 1: print("Accuracy: ",PLR_Model_1.score(X_test, y_test))

### Ridge regression

In [ ]:
##### RIDGE #####

# Use the best regularization strength parameter
# optima parameter indsat
PLR_Model_Ridge = LogisticRegression()
#penalty = 'l2', 
                           # random_state = seed_number,
                            #solver = 'newton-cg', 
                            #C=0.0017, 
                            ##class_weight="balanced"
####################################################################################################
                                        # grid search
# ####################################################################################################

parameters = {
    'C':np.logspace(-3,3,50),
   'solver': ['newton-cg', 'lbfgs', 'sag' and 'saga'],
     'penalty' : ['l2']
            }
PLR_model_Ridge_Grid = GridSearchCV(    PLR_Model_Ridge, 
                                         parameters, 
                                         cv=5, 
                                         scoring = "roc_auc", 
                                         # verbose=10, 
                                         n_jobs = -1
                                         )
PLR_model_Ridge_Grid.fit(X_train, y_train)
print(sorted(PLR_model_Ridge_Grid.cv_results_.keys()))

PLR_Model_Ridge = PLR_model_Ridge_Grid
# ####################################################################################################
# ####################################################################################################

# Training the Random Forest model
PLR_Model_Ridge.fit(X_train, y_train)
# print(datetime.now() - t12)

print(PLR_Model_Ridge.best_estimator_)
print("Tuned Hyperparameters :", PLR_Model_Ridge.best_params_)

# Model accuracy
if print_flag == 1: print("Accuracy: ",PLR_Model_Ridge.score(X_test, y_test))

# Model prediction
PLR_Model_Ridge_preds = PLR_Model_Ridge.predict(X_test)
# Probability predicting for ROC plot
PLR_Model_Ridge_probs = PLR_Model_Ridge.predict_proba(X_test)[:,1]
# Model prediction
PLR_Model_Ridge_preds = np.where(PLR_Model_Ridge_probs > 0.5, 1, 0)

# Used for confusion matrix
y_prediction = dict()
y_prediction['status'] = y_test.copy()
y_prediction["Preds"] = PLR_Model_Ridge_preds

# Make confusion matrix
confusion_matrixs = pd.crosstab(y_prediction['status'], y_prediction["Preds"], rownames=['Actual'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrixs)

# Classification report with f1, recall, precision and accuract of the model
Class_DF = classification_report(y_test, PLR_Model_Ridge_preds,output_dict = True)
if print_flag == 1: print("Classification parameters: \n",pd.DataFrame(Class_DF))

# # Probability predicting for ROC plot
# PLR_Model_Ridge_probs = PLR_Model_Ridge.predict_proba(X_test)[:,1]

# ROC plot
## calculate fpr and tpr to use in ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, PLR_Model_Ridge_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 10)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Ridge Logistic Regression"])
os.system('say "Ridge log is done."')

### Lasso regression

In [ ]:
###### LASSO ######

# Use the best regularization strength parameter
# optima parameter indsat
PLR_Model_Lasso = LogisticRegression(penalty = 'l1', 
                            random_state = seed_number,
                            solver = 'liblinear', 
                            C=0.089, 
                            # class_weight="balanced"
                            )
'liblinear' and 'saga' 
# ####################################################################################################
#                                         # grid search
# ####################################################################################################

parameters = {
    'C':np.arange(0,10,0.1),
   'solver': ['newton-cg', 'lbfgs', 'liblinear','sag' and 'saga'],
     'penalty' : ['l1']
            }
PLR_model_Lasso_Grid = GridSearchCV(          PLR_Model_Lasso, 
                                         parameters, 
                                         cv=5, 
                                         scoring = "roc_auc", 
                                         # verbose=10, 
                                         n_jobs = -1
                                         )
PLR_model_Lasso_Grid.fit(X_train, y_train)
#print(sorted(PLR_model_Grid.cv_results_.keys()))

PLR_Model_Lasso = PLR_model_Lasso_Grid
# ####################################################################################################
# ###################################################################################################

# Training the Random Forest model
PLR_Model_Lasso.fit(X_train, y_train)
# print(datetime.now() - t12)

print(PLR_Model_Lasso.best_estimator_)
print("Tuned Hyperparameters :", PLR_Model_Lasso.best_params_)

# Model accuracy
if print_flag == 1: print("Accuracy: ",PLR_Model_Lasso.score(X_test, y_test))

    # Model prediction
PLR_Model_Lasso_preds = PLR_Model_Lasso.predict(X_test)
# Probability predicting for ROC plot
PLR_Model_Lasso_probs = PLR_Model_Lasso.predict_proba(X_test)[:,1]
# Model prediction
PLR_Model_Lasso_preds = np.where(PLR_Model_Lasso_probs > 0.35, 1, 0)

#PLR_Model_Lasso.coef_    # dense np.array



# Used for confusion matrix
y_prediction = dict()
y_prediction['dropouts'] = y_test.copy()
y_prediction["Preds"] = PLR_Model_Lasso_preds

# Make confusion matrix
confusion_matrixs = pd.crosstab(y_prediction['dropouts'], y_prediction["Preds"], rownames=['Actual'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix: \n",confusion_matrixs)

# Classification report with f1, recall, precision and accuract of the model
Class_DF = classification_report(y_test, PLR_Model_Lasso_preds,output_dict = True)
if print_flag == 1: print("Classification parameters: \n",pd.DataFrame(Class_DF))

# # Probability predicting for ROC plot
PLR_Model_Lasso_probs = PLR_Model_Lasso.predict_proba(X_test)[:,1]
#print(datetime.now() - t1)

# ROC plot
## calculate fpr and tpr to use in ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, PLR_Model_Lasso_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 10)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Lasso Logistic Regression"])
os.system('say "Lasso log is done."')

## Random forest

In [ ]:
#t1 = datetime.now()
#print(t1)
# function specifications
plot_flag = 1
print_flag = 1
#DF = Transposed_DF

## Random Forest model specified
# Her kan man regulere om det er balanced eller ej
# Har intastet de bedste parametre
# RF_Model = RandomForestClassifier(  
#                                     # n_estimatorsint = 
#                                     max_depth = 80, 
#                                     random_state = seed_number,
#                                     bootstrap = True,
#                                     criterion = 'entropy', 
#                                     max_features = 54,
#                                     # max_features = "sqrt",
#                                     n_estimators = 300,
#                                 )
# for balanceret                                
RF_Model = RandomForestClassifier(  
                                    # n_estimatorsint = 
                                    #max_depth = 33, 
                                    random_state = seed_number,
                                    bootstrap = True,
                                    oob_score = True,
                                    #criterion = 'entropy', 
                                    #max_features = 80,
                                    # max_features = "sqrt",
                                    #n_estimators = 600,
                                    #class_weight = "balanced"
                                )
# Splitting dataset into X,y sets                                
#X = DF.drop(["dropouts"], axis = 1)
#y = DF["dropouts"]

# Split data into test and train
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed_number)

# ####################################################################################################
#                                         # grid search
# ####################################################################################################
#t12 = datetime.now()
# print(t12)
#n_variables = len(Transposed_DF.columns)
parameters = {
     'criterion': ['entropy'],
     'max_depth':[10],
     #'max_features': ['sqrt'],
     'max_features': [8],
     'n_estimators': [1200],
     #'min_samples_split'
     #'min_samples_leaf'
     # max_leaf_nodes
     # min_impurity_decrease
     # max_samples
             }
RF_Model_Grid = GridSearchCV(    RF_Model, 
                                         parameters, 
                                         cv=5, scoring = "roc_auc", 
                                         verbose=10, 
                                        # n_jobs = -1
                                         )
RF_Model_Grid.fit(X_train, y_train)
print(sorted(RF_Model_Grid.cv_results_.keys()))
#print(datetime.now() - t12)

RF_Model = RF_Model_Grid
os.system('say "Gridsearch is done."')
# ####################################################################################################
# ####################################################################################################

# Training the Random Forest model
#RF_Model.fit(X_train, y_train)


# print(RF_Model.best_estimator_)

# Model accuracy
if print_flag == 1: print("Accuracy: ",RF_Model.score(X_test, y_test))

# Model prediction
RF_Model_preds = RF_Model.predict(X_test)
# Probability predicting for ROC plot
RF_Model_probs = RF_Model.predict_proba(X_test)[:,1]
# Model prediction
RF_Model_preds = np.where(RF_Model_probs > 0.5, 1, 0)
# Used for confusion matrix
y_prediction = dict()
y_prediction['dropouts'] = y_test.copy()
y_prediction["Preds"] = RF_Model_preds

# Make confusion matrix
confusion_matrixs = pd.crosstab(y_prediction['dropouts'], y_prediction["Preds"], rownames=['Actual'], colnames=['Predicted'])
if print_flag == 1: print("Confusion Matrix:\n",confusion_matrixs)

# Classification report with f1, recall, precision and accuract of the model
Class_DF = classification_report(y_test, RF_Model_preds,output_dict = True)
if print_flag == 1: print("Classification parameters:\n",pd.DataFrame(Class_DF))

# Probability predicting for ROC plot
RF_Model_probs = RF_Model.predict_proba(X_test)[:,1]

# ROC plot
## calculate fpr and tpr to use in ROC plot
if plot_flag == 1:
    fpr, tpr, threshold = metrics.roc_curve(y_test, RF_Model_probs)
    roc_auc = round(metrics.auc(fpr, tpr), 4)
    rates_list = [[tpr, fpr, roc_auc]]
    ## Creates the ROC plot
    roc_plot(values = rates_list, colors=['blue'], lab = ["Random Forrest"])
os.system('say "Random Fores is done."')

### Variable importance

In [ ]:
rf = RandomForestClassifier(  max_depth = 10, 
                                    random_state = seed_number,
                                    bootstrap = True,
                                    criterion = 'entropy', 
                                    max_features = 8,
                                    # max_features = "sqrt",
                                    n_estimators = 1200,
                                    oob_score = True
                                    #class_weight = "balanced"
                                )
rf.fit(X_train, y_train)

rf.feature_importances_

rf.oob_score_

import numpy as np
 
importances = rf.feature_importances_
#
# Sort the feature importance in descending order
#
sorted_indices = np.argsort(importances)[::-1]
 
feat_labels = dummy_df.columns[1:]
 
for f in range(X_train.shape[1]):
    print("%2d) %-*s %f" % (f + 1, 30,
                            feat_labels[sorted_indices[f]],
                            importances[sorted_indices[f]]))

## Neural network

### Neural network model 1